In [1]:
import os
import pandas as pd
import numpy as np
import random

def dataset_movielens ():
    data_dir="Movie-Lens-dataset"
    # my_data = pd.read_csv(r"Movie-Lens-dataset\u.data", '\t', names=['user_id', 'item_id', 'rating', 'timestamp'],engine='python')

    names = ['user_id', 'item_id', 'rating', 'timestamp']
    movie_db = pd.read_csv(os.path.join(data_dir, 'u.data'), '\t', names=names,
                           engine='python')
    #creating time intervals
    movie_db['date'] = pd.to_datetime(movie_db['timestamp'].astype(int), unit='s')
    # movie_db['date1'] = pd.to_datetime(movie_db['date'],format="%m/%d/%Y  %H:%M:%S").dt.strftime('%Y-%m-%d  %H:%M:%S')

    # print(movie_db)

    item_info=movie_db.groupby('item_id', as_index=False).rating.agg({np.mean, np.var})
    item_info.reset_index(level=0, inplace=True)

    # inject fake ratings
    fake_db = pd.DataFrame( columns=['user_id', 'item_id', 'rating', 'date'])
    filler_db=pd.DataFrame(columns=['user_id', 'item_id', 'rating', 'date'])
    target_db=pd.DataFrame(columns=['user_id', 'item_id', 'rating', 'date'])
    selected_db=pd.DataFrame(columns=['user_id', 'item_id', 'rating', 'date'])
    fake_profile=[]
    filler_size= 0.05 # from 1% to 10% which is 1% of items excluding target items
    attack_size= 0.15
    target_size=100
    selected_item_size=20
    attack_date=random.sample(list(movie_db['date'].unique()),1) 
    #     print('attack day is: ',attack_date)
    #'1998-04-19'
    #select day randomly, include all days of 8 months
    x=0.01 ## percentage of fillers for AOP attack
    attack_model='bandwagon' ## can be filled with one of : (random, average, bandwagon,AOP)
    avg_rating=movie_db['rating'].mean() 
    std_rating=movie_db['rating'].std() 
    
    # defining fake profiles
    for i in range (round(attack_size*movie_db['user_id'].nunique())):
        fake_profile.append(movie_db['user_id'].max()+(i+1))
#     print('number of fake profiles generated is: \n',round(attack_size*movie_db['user_id'].nunique()))

    #defining target items
    #30 items are choosen randomly out of 1682 items
    target_items=random.sample(list(movie_db['item_id'].unique()),target_size)
    #     print('number of target items selected is: \n',target_size)
    #defining filler items
    fillers = [x for x in list(movie_db['item_id']) if x not in target_items]
    #randomly choosed filler items based on filler size
    filler_items=random.sample(fillers,round(filler_size*len(set(fillers))))

    #insert ratings for filler items
    filler_db.drop(filler_db.index, inplace=True)
    for i in filler_items:
        if attack_model=='random':
            filler_db=filler_db.append(pd.DataFrame({'user_id': 0, 'item_id': i ,'rating':round(np.random.normal(avg_rating, std_rating),2),'date':[attack_date[0]]}), ignore_index=True)
        if attack_model=='average':
            filler_db=filler_db.append(pd.DataFrame({'user_id': 0, 'item_id': i ,'rating':item_info.loc[item_info['item_id']==i,'mean'],'date':[attack_date[0]]}), ignore_index=True)
        if attack_model=='bandwagon':
            filler_db=filler_db.append(pd.DataFrame({'user_id': 0, 'item_id': i ,'rating':round(np.random.normal(avg_rating, std_rating),2),'date':[attack_date[0]]}), ignore_index=True)
        if attack_model=='AOP':
            AOP_filler_db=pd.DataFrame(columns=['user_id', 'item_id', 'rating', 'date'])
            AOP_fillers=movie_db.groupby('item_id', as_index=False)['rating'].count()
            AOP_fillers=AOP_fillers.sort_values('rating',ascending=False)
            x=0.01 ## percentage of fillers for AOP attack
            AOP_fillers=AOP_fillers.head(int(len(AOP_fillers)*(x)))
            AOP_fillers=[x for x in AOP_fillers['item_id']]
#             if attack_model=='AOP':
            for i in AOP_fillers:
                AOP_filler_db=AOP_filler_db.append(pd.DataFrame({'user_id':random.sample(fake_profile,1), 'item_id': i ,'rating':np.random.normal(item_info.loc[item_info['item_id']==i,'mean'],item_info.loc[item_info['item_id']==i,'var']),'date1':[attack_date[0]]}), ignore_index=True)
            filler_db=AOP_filler_db

#             print('number of filler items selected for AOP attack is: \n',round(len(set(AOP_fillers))))
#         else:
#             print('number of filler items selected is: \n',round(filler_size*len(set(fillers))))


    #insert ratings for target items
    target_db.drop(target_db.index, inplace=True)
    for i in target_items:
        #print(random.sample(fake_profile,1),i,5)
        target_db=target_db.append(pd.DataFrame({'user_id':0,'item_id':i,'rating':5,'date':[attack_date[0]]}),ignore_index=True)

    injected_db=pd.DataFrame()
    injected_db=injected_db.append(target_db)
    injected_db=injected_db.append(filler_db)

    injected_db
    movie_db=movie_db.append(injected_db)

    movie_db.fillna(0, inplace=True)
    item_db=movie_db.groupby(['item_id','date'], as_index=False).rating.agg({np.sum})

    item_db.index
    item_db=item_db.reset_index(level=['item_id', 'date'])
    item_db.columns = ['item_id', 'date','rating']

    item_db=item_db.pivot(index='item_id',columns='date',values='rating')
    item_db.fillna(0, inplace=True)
    item_db=item_db.reset_index(level=['item_id'])
    item_db['target']=0
    item_db.loc[item_db.item_id.isin(target_items),'target']='1'
    y=item_db[['target']]
    x=item_db.drop('target', axis=1)
    x.set_index('item_id', inplace=True)
    x_column_names=x.columns
    x.index.names = ['item_id']
    y.index.names = ['item_id']
    from sklearn import preprocessing

#     x1 = x.values #returns a numpy array
#     min_max_scaler = preprocessing.MinMaxScaler(feature_range=(0, 5))
#     x_scaled = min_max_scaler.fit_transform(x1)
#     x_scaled = pd.DataFrame(x_scaled,columns=x_column_names)
#     x_scaled.index.names = ['item_id']
    return (x,y)



In [2]:
def dataset_movielens_daily (target_size_var,filler_size_var,attack_size_var,attack_model_var):
    data_dir="Movie-Lens-dataset"
    # my_data = pd.read_csv(r"Movie-Lens-dataset\u.data", '\t', names=['user_id', 'item_id', 'rating', 'timestamp'],engine='python')

    names = ['user_id', 'item_id', 'rating', 'timestamp']
    movie_db = pd.read_csv(os.path.join(data_dir, 'u.data'), '\t', names=names,
                           engine='python')
    #creating time intervals
    movie_db['date'] = pd.to_datetime(movie_db['timestamp'].astype(int), unit='s')
    movie_db['date1'] = pd.to_datetime(movie_db['date'],format="%m/%d/%Y").dt.strftime('%Y-%m-%d')

    # print(movie_db)

    item_info=movie_db.groupby('item_id', as_index=False).rating.agg({np.mean, np.var})
    item_info.reset_index(level=0, inplace=True)

    # inject fake ratings
    fake_db = pd.DataFrame( columns=['user_id', 'item_id', 'rating', 'date1'])
    filler_db=pd.DataFrame(columns=['user_id', 'item_id', 'rating', 'date1'])
    target_db=pd.DataFrame(columns=['user_id', 'item_id', 'rating', 'date1'])
    selected_db=pd.DataFrame(columns=['user_id', 'item_id', 'rating', 'date1'])
    fake_profile=[]
    filler_size= filler_size_var #0.05 # from 1% to 10% which is 1% of items excluding target items
    attack_size= attack_size_var  #0.15
    target_size=target_size_var#100
    selected_item_size=20
    attack_date=random.sample(list(movie_db['date1'].unique()),1) 
    #     print('attack day is: ',attack_date)
    #'1998-04-19'
    #select day randomly, include all days of 8 months
    x=0.01 ## percentage of fillers for AOP attack
    attack_model=attack_model_var  #'bandwagon' ## can be filled with one of : (random, average, bandwagon,AOP)
    avg_rating=movie_db['rating'].mean() 
    std_rating=movie_db['rating'].std() 
    
    # defining fake profiles
    for i in range (round(attack_size*movie_db['user_id'].nunique())):
        fake_profile.append(movie_db['user_id'].max()+(i+1))
        
    #defining target items
    #30 items are choosen randomly out of 1682 items
    target_items=random.sample(list(movie_db['item_id'].unique()),target_size)
    #     print('number of target items selected is: \n',target_size)
    #defining filler items
    fillers = [x for x in list(movie_db['item_id']) if x not in target_items]
    #randomly choosed filler items based on filler size
    filler_items=random.sample(fillers,round(filler_size*len(set(fillers))))

    #insert ratings for filler items
    filler_db.drop(filler_db.index, inplace=True)
    for i in filler_items:
        if attack_model=='random':
            filler_db=filler_db.append(pd.DataFrame({'user_id': 0, 'item_id': i ,'rating':round(np.random.normal(avg_rating, std_rating),2),'date':[attack_date[0]]}), ignore_index=True)
        if attack_model=='average':
            filler_db=filler_db.append(pd.DataFrame({'user_id': 0, 'item_id': i ,'rating':item_info.loc[item_info['item_id']==i,'mean'],'date':[attack_date[0]]}), ignore_index=True)
        if attack_model=='bandwagon':
            filler_db=filler_db.append(pd.DataFrame({'user_id': 0, 'item_id': i ,'rating':round(np.random.normal(avg_rating, std_rating),2),'date':[attack_date[0]]}), ignore_index=True)
        if attack_model=='AOP':
            AOP_filler_db=pd.DataFrame(columns=['user_id', 'item_id', 'rating', 'date1'])
            AOP_fillers=movie_db.groupby('item_id', as_index=False)['rating'].count()
            AOP_fillers=AOP_fillers.sort_values('rating',ascending=False)
            x=0.01 ## percentage of fillers for AOP attack
            AOP_fillers=AOP_fillers.head(int(len(AOP_fillers)*(x)))
            AOP_fillers=[x for x in AOP_fillers['item_id']]
#             if attack_model=='AOP':
            for i in AOP_fillers:
                AOP_filler_db=AOP_filler_db.append(pd.DataFrame({'user_id':random.sample(fake_profile,1), 'item_id': i ,'rating':np.random.normal(item_info.loc[item_info['item_id']==i,'mean'],item_info.loc[item_info['item_id']==i,'var']),'date1':[attack_date[0]]}), ignore_index=True)
            filler_db=AOP_filler_db

#             print('number of filler items selected for AOP attack is: \n',round(len(set(AOP_fillers))))
#         else:
#             print('number of filler items selected is: \n',round(filler_size*len(set(fillers))))

    #insert ratings for target items
    target_db.drop(target_db.index, inplace=True)
    for i in target_items:
        #print(random.sample(fake_profile,1),i,5)
        target_db=target_db.append(pd.DataFrame({'user_id':0,'item_id':i,'rating':5,'date':[attack_date[0]]}),ignore_index=True)

    injected_db=pd.DataFrame()
    injected_db=injected_db.append(target_db)
    injected_db=injected_db.append(filler_db)

    injected_db
    movie_db=movie_db.append(injected_db)

    movie_db.fillna(0, inplace=True)
    item_db=movie_db.groupby(['item_id','date1'], as_index=False).rating.agg({np.sum})

    item_db.index
    item_db=item_db.reset_index(level=['item_id', 'date1'])
    item_db.columns = ['item_id', 'date','rating']

    item_db=item_db.pivot(index='item_id',columns='date',values='rating')
    item_db.fillna(0, inplace=True)
    item_db=item_db.reset_index(level=['item_id'])
    item_db['target']=0
    item_db.loc[item_db.item_id.isin(target_items),'target']='1'
    y=item_db[['target']]
    x=item_db.drop('target', axis=1)
    x.set_index('item_id', inplace=True)
    x_column_names=x.columns
    x.index.names = ['item_id']
    y.index.names = ['item_id']
    from sklearn import preprocessing

#     x1 = x.values #returns a numpy array
#     min_max_scaler = preprocessing.MinMaxScaler(feature_range=(0, 5))
#     x_scaled = min_max_scaler.fit_transform(x1)
#     x_scaled = pd.DataFrame(x_scaled,columns=x_column_names)
#     x_scaled.index.names = ['item_id']
    return (x,y)



In [2]:
def dataset_Netflix_daily (target_size_var,filler_size_var,attack_size_var,attack_model_var):
    import random

    
    movie_db = pd.read_csv('C:/Users/Mahsa/Shilling_Attack_Detection_SNN/Netflix-dataset/netflix_sample.csv', names = ['user_id','rating','date','item_id'],header = None)
#     movie_db = pd.read_csv('C:/Users/Mahsa/Shilling_Attack_Detection_SNN/Netflix-dataset/netflix_sample_complete.csv',skiprows=1, names = ['user_id','rating','date','item_id'],header = None)

    selected_item_size=20
    x=0.01 ## percentage of fillers for AOP attack
    #attack_model='average' ## can be filled with one of : (random, average, bandwagon,AOP)
    # avg_rating=movie_db['rating'].mean() 
    # std_rating=movie_db['rating'].std()  
    names = ['user_id', 'item_id', 'rating', 'timestamp']

    #creating time intervals
    # movie_db['date'] = pd.to_datetime(movie_db['timestamp'].astype(int), unit='s')
    movie_db['date1'] = movie_db['date'] #pd.to_datetime(movie_db['date'],format="%m/%d/%Y").dt.strftime('%Y-%m-%d')

    # print(movie_db)

    item_info=movie_db.groupby('item_id', as_index=False).rating.agg({np.mean, np.var})
    item_info.reset_index(level=0, inplace=True)

    # inject fake ratings
    fake_db = pd.DataFrame( columns=['user_id', 'item_id', 'rating', 'date1'])
    filler_db=pd.DataFrame(columns=['user_id', 'item_id', 'rating', 'date1'])
    target_db=pd.DataFrame(columns=['user_id', 'item_id', 'rating', 'date1'])
    selected_db=pd.DataFrame(columns=['user_id', 'item_id', 'rating', 'date1'])
    fake_profile=[]
    filler_size= filler_size_var #0.05 # from 1% to 10% which is 1% of items excluding target items
    attack_size= attack_size_var  #0.15
    target_size=target_size_var#100
    selected_item_size=20
    attack_date=random.sample(list(movie_db['date1'].unique()),1) 
    #     print('attack day is: ',attack_date)
    #'1998-04-19'
    #select day randomly, include all days of 8 months
    x=0.01 ## percentage of fillers for AOP attack
    attack_model=attack_model_var  #'bandwagon' ## can be filled with one of : (random, average, bandwagon,AOP)
    avg_rating=movie_db['rating'].mean() 
    std_rating=movie_db['rating'].std() 

    # defining fake profiles
    for i in range (round(attack_size*movie_db['user_id'].nunique())):
        fake_profile.append(movie_db['user_id'].max()+(i+1))

    #defining target items
    #30 items are choosen randomly out of 1682 items
    target_items=random.sample(list(movie_db['item_id'].unique()),target_size)

    #     print('number of target items selected is: \n',target_size)
    #defining filler items
    fillers = [x for x in list(movie_db['item_id']) if x not in target_items]
    #randomly choosed filler items based on filler size
    filler_items=random.sample(fillers,round(filler_size*len(set(fillers))))

    #insert ratings for filler items
    filler_db.drop(filler_db.index, inplace=True)
    for i in filler_items:
        if attack_model=='random':
            filler_db=filler_db.append(pd.DataFrame({'user_id': 0, 'item_id': i ,'rating':round(np.random.normal(avg_rating, std_rating),2),'date':[attack_date[0]]}), ignore_index=True)
        if attack_model=='average':
            filler_db=filler_db.append(pd.DataFrame({'user_id': 0, 'item_id': i ,'rating':item_info.loc[item_info['item_id']==i,'mean'],'date':[attack_date[0]]}), ignore_index=True)
        if attack_model=='bandwagon':
            filler_db=filler_db.append(pd.DataFrame({'user_id': 0, 'item_id': i ,'rating':round(np.random.normal(avg_rating, std_rating),2),'date':[attack_date[0]]}), ignore_index=True)
        if attack_model=='AOP':
            AOP_filler_db=pd.DataFrame(columns=['user_id', 'item_id', 'rating', 'date1'])
            AOP_fillers=movie_db.groupby('item_id', as_index=False)['rating'].count()
            AOP_fillers=AOP_fillers.sort_values('rating',ascending=False)
            x=0.01 ## percentage of fillers for AOP attack
            AOP_fillers=AOP_fillers.head(int(len(AOP_fillers)*(x)))
            AOP_fillers=[x for x in AOP_fillers['item_id']]
    #             if attack_model=='AOP':
            for i in AOP_fillers:
                AOP_filler_db=AOP_filler_db.append(pd.DataFrame({'user_id':random.sample(fake_profile,1), 'item_id': i ,'rating':np.random.normal(item_info.loc[item_info['item_id']==i,'mean'],item_info.loc[item_info['item_id']==i,'var']),'date1':[attack_date[0]]}), ignore_index=True)
            filler_db=AOP_filler_db

    #             print('number of filler items selected for AOP attack is: \n',round(len(set(AOP_fillers))))
    #         else:
    #             print('number of filler items selected is: \n',round(filler_size*len(set(fillers))))

    #insert ratings for target items
    target_db.drop(target_db.index, inplace=True)
    for i in target_items:
        #print(random.sample(fake_profile,1),i,5)
        target_db=target_db.append(pd.DataFrame({'user_id':0,'item_id':i,'rating':5,'date':[attack_date[0]]}),ignore_index=True)

    injected_db=pd.DataFrame()
    injected_db=injected_db.append(target_db)
    injected_db=injected_db.append(filler_db)

    injected_db
    movie_db=movie_db.append(injected_db)

    movie_db.fillna(0, inplace=True)
    item_db=movie_db.groupby(['item_id','date1'], as_index=False).rating.agg({np.sum})

    item_db.index
    item_db=item_db.reset_index(level=['item_id', 'date1'])
    item_db.columns = ['item_id', 'date','rating']

    item_db=item_db.pivot(index='item_id',columns='date',values='rating')
    item_db.fillna(0, inplace=True)
    item_db=item_db.reset_index(level=['item_id'])
    item_db['target']=0
    item_db.loc[item_db.item_id.isin(target_items),'target']='1'
    y=item_db[['target']]
    x=item_db.drop('target', axis=1)
    x.set_index('item_id', inplace=True)
    x_column_names=x.columns
    x.index.names = ['item_id']
    y.index.names = ['item_id']

    return (x,y)

